In [16]:
# 사전 설치 : pip install ollama
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
import ollama
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import WikipediaRetriever
import warnings
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage
import gradio as gr
import tempfile

In [17]:
qa_chain = None
embeddings = OllamaEmbeddings(model="gemma2")
llm = ChatOllama(model = "gemma2", temperature=0.6, verbose=False)

In [18]:
def process_file(file_path):
    """파일을 처리하고 QA 체인을 설정하는 함수"""
    global qa_chain
    
    try:
        # 직접 파일 경로를 TextLoader에 전달
        loader = TextLoader(file_path, encoding='UTF8')
        documents = loader.load()
        
        # 벡터 저장소 생성
        vectorstore = FAISS.from_documents(documents, embeddings)
        
        # QA 체인 설정
        retriever = vectorstore.as_retriever()
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=retriever
        )
        
        return "파일이 성공적으로 처리되었습니다."
    
    except Exception as e:
        return f"파일 처리 중 오류가 발생했습니다: {str(e)}"

In [19]:
def chat(message, history):
    """챗봇 응답을 생성하는 함수"""
    global qa_chain
    
    if qa_chain is None:
        return "먼저 파일을 업로드해 주세요."
    
    try:
        # QA 체인으로 응답 생성
        response = qa_chain.run(message)
        return response
    
    except Exception as e:
        return f"응답 생성 중 오류가 발생했습니다: {str(e)}"


In [20]:
# Gradio 인터페이스 설정
with gr.Blocks() as demo:
    gr.Markdown("# RAG Chatbot")
    
    with gr.Row():
        file_output = gr.Textbox(label="파일 처리 상태")
        upload_button = gr.File(
            label="텍스트 파일 업로드",
            file_types=[".txt"]
        )
    
    chatbot_interface = gr.ChatInterface(
        chat,
        examples=[
            "고조선은 언제 설립되었나요?",
            "삼국시대의 주요 사건은 무엇인가요?",
            "조선시대의 대표적인 문화유산은 무엇인가요?"
        ]
    )
    
    upload_button.upload(
        fn=process_file,
        inputs=[upload_button],
        outputs=[file_output]
    )

c:\Ai_Project\.venv\lib\site-packages\gradio\components\chatbot.py:248: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


In [21]:
demo.launch(server_port=7861, server_name="0.0.0.0")

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


C:\Users\human-14\AppData\Local\Temp\ipykernel_13256\4268912487.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(message)


In [15]:
demo.close()

Closing server running on port: 7861
